In [1]:
import torch
import pandas as pd

from algorithms import LogisticRegressionRBF
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

x, y = make_classification(n_samples=1000, n_features=3, n_informative=3, n_redundant=0, random_state=1)
x = torch.FloatTensor(x)
y = torch.FloatTensor(y)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)

In [10]:
model = LogisticRegressionRBF(x_train, rbf='gaussian')
model = model.fit(x_train, y_train, 1000)

Epoch:     1 | CrossEntropyLoss:  0.69440
Epoch:     3 | CrossEntropyLoss:  0.66335
Epoch:     6 | CrossEntropyLoss:  0.61960
Epoch:    10 | CrossEntropyLoss:  0.56666
Epoch:    18 | CrossEntropyLoss:  0.47878
Epoch:    30 | CrossEntropyLoss:  0.38565
Epoch:    50 | CrossEntropyLoss:  0.29767
Epoch:    83 | CrossEntropyLoss:  0.23171
Epoch:   138 | CrossEntropyLoss:  0.18508
Epoch:   226 | CrossEntropyLoss:  0.15188
Epoch:   372 | CrossEntropyLoss:  0.12612
Epoch:   610 | CrossEntropyLoss:  0.10682
Epoch:  1000 | CrossEntropyLoss:  0.09297


## Testing

In [11]:
metrics = model.metrics_tab(x_test, y_test, metric='by_roc')
tab = pd.DataFrame({'0': metrics['0.0'], '1': metrics['1.0']}).T.round(3)
print(f'Accuracy: {metrics["accuracy"]: 0.2%} | Threshold: {metrics["threshold"]: 0.2f}')
tab

Accuracy:  97.67% | Threshold:  0.47


,precision,recall,f1-score,support
0,0.987,0.967,0.977,153.0
1,0.967,0.986,0.976,147.0


In [12]:
from metrics import roc_curve_plot, auc_roc

y_prob = model(x_test).detach().cpu().numpy().flatten()
print(f'AUC-ROC: {auc_roc(y_test, y_prob)}')
roc_curve_plot(y_test, y_prob, fill=True)

AUC-ROC: 0.9881730469965764


## Visualization of initial data

In [13]:
from sklearn.manifold import TSNE
import plotly.express as px

colors = list(map(lambda e: str(int(e)), y))

if x.shape[1] <= 2:
    fig = px.scatter(x=x[:, 0], y=x[:, 1], title='<b>Initial Distribution</b>', color=colors)

else:
    tsne_x = TSNE().fit_transform(x)
    fig = px.scatter(x=tsne_x[:, 0], y=tsne_x[:, 1], title='<b>TSNE of Distribution</b>', color=colors)

fig.show()

In [14]:
phi = model.make_phi_matrix(x)

if phi.shape[1] <= 2:
    fig = px.scatter(x=phi[:, 0], y=phi[:, 1], title='<b>Distribution after making Phi-Matrix</b>', color=colors)

else:
    tsne_phi = TSNE().fit_transform(phi)
    fig = px.scatter(x=tsne_phi[:, 0], y=tsne_phi[:, 1], title='<b>TSNE of Distribution after making Phi-Matrix</b>', color=colors)

fig.show()